In [1]:
!pip install ale_py

import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
import ale_py

from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.evaluation import evaluate_policy
import matplotlib.pyplot as plt
from gymnasium.wrappers import RecordVideo
import torch
import torch.nn as nn
from tqdm import tqdm
%matplotlib inline


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.8 MB/s eta 0:00:00a 0:00:01


/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/usr/local/lib/python3.10/dist-packages/shimmy/registration.py", line 304, in register_gymnasium_envs
    _register_atari_envs()
  File "/usr/local/lib/python3.10/dist-packages/shimmy/registration.py", line 205, in _register_atari_envs
    import ale_py
  File "/usr/local/lib/python3.10/dist-packages/ale_py/__init__.py", line 68, in <module>
    register_v0_v4_envs()
  File "/usr/local/lib/python3.10/dist-packages/ale_py/registration.py", line 178, in register_v0_v4_envs
    _register_rom_configs(legacy_games, obs_types, versions)
  File "/usr/local/lib/python3.10/dist-packages/ale_py/registration.py", line 63, in _register_rom_configs
    gymnasium.register(
AttributeError:

In [3]:
TRAIN_STEPS = 1000000
EVAL_EPISODES = 10

In [4]:
gym.register_envs(ale_py)
#env = gym.make('ALE/Pacman-v5', max_episode_steps=1000)

train_env = SubprocVecEnv([lambda: gym.make('ALE/Pacman-v5', max_episode_steps=1000) for _ in range(16)], start_method="fork")
eval_env = gym.make('ALE/Pacman-v5', max_episode_steps=5000)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [5]:
train_env.reset()
model = DQN("CnnPolicy", train_env, verbose=0, buffer_size=100000, tensorboard_log='/kaggle/working/tensorboard_logs/')

with tqdm(total=TRAIN_STEPS) as pbar:
    def callback(_locals, _globals):
        pbar.update(_locals['self'].num_timesteps - pbar.n)
        return True

    model.learn(total_timesteps=TRAIN_STEPS, tb_log_name="first_run", callback=callback)




100%|██████████| 1000000/1000000 [19:32<00:00, 852.55it/s]


NameError: name 'EVAL_EPS' is not defined

In [6]:
model.save("dqn_stable_baselines")
mean_reward, _ = evaluate_policy(model, eval_env, n_eval_episodes=EVAL_EPISODES)
train_env.close()
print(mean_reward)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


47.5


In [7]:
model = DQN.load("dqn_stable_baselines")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:437: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via th

In [9]:
eval_env = gym.make('ALE/Pacman-v5', max_episode_steps=5000, render_mode="rgb_array")
eval_env = RecordVideo(eval_env, 'video')
eval_env = DummyVecEnv([lambda: eval_env])

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:297: UserWarning: WARN: env.is_vector_env to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.is_vector_env` for environment variables or `env.get_attr('is_vector_env')` that will search the reminding wrappers.
  logger.warn(


In [10]:
mean_reward, _ = evaluate_policy(model, eval_env, n_eval_episodes=1)

Moviepy - Building video /kaggle/working/video/rl-video-episode-0.mp4.
Moviepy - Writing video /kaggle/working/video/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /kaggle/working/video/rl-video-episode-0.mp4


In [11]:
print(mean_reward)

48.0


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
